<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Data extraction solution for Mariana's research project

## Extract data with an Amazon EMR Apache Spark cluster

## Year 2020, month 12

### Import the required libraries

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, col, array_contains

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1720441205821_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Load data into an Amazon EMR Apache Spark DataFrame

The data corresponding to month 12 of year 2022 is malformed. While the data samples of other months have around 37 columns, the data of month 12 of year 2023 has only 3 columns, which makes it unsuitable.

In [2]:
# Set the S3 bucket and folder paths
data_source = 's3://gelctweets2022/2023_12/'
output_path = 's3://gelcawsemr2022/2022_12/filtered_tweets.jsonl'

# Create a SparkSession
spark = SparkSession.builder.appName('The Twitter Grab Corpus').getOrCreate()

# Read the JSONL files into a DataFrame
#tweets_spark_df = spark.read.json(args.data_source) # RevA parameters
tweets_spark_df = spark.read.option('recursiveFileLookup', 'true').json(data_source) # RevB parameters

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Show the first few rows of the DataFrame
tweets_spark_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                data|              errors|            includes|
+--------------------+--------------------+--------------------+
|{{NULL, NULL}, 15...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 30...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 12...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 23...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 24...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 27...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 42...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 14...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 48...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 12...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 14...|[{Sorry, you are ...|{NULL, NULL, NULL...|
|{{NULL, NULL}, 86...|                NULL|{NULL, NULL, NULL...|
|{{NULL, NULL}, 12...|   

In [4]:
# Show the quantity of columns of the DataFrame
len(tweets_spark_df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

In [5]:
# Show the quantity of rows (tweets) of the DataFrame
tweets_spark_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

69687999

### Filter the dataframe

In [6]:
# Define the list of hashtags for DataFrame filtering
hashtags = [
    'acolhimento', 
    'Acolhimento', 
    'ACOLHIMENTO', 
    'aporofobia', 
    'Aporofobia', 
    'APOROFOBIA', 
    'brasilvaivirarvenezuela', 
    'Brasilvaivirarvenezuela', 
    'BrasilVaiVirarVenezuela', 
    'BRASILVAIVIRARVENEZUELA', 
    'crisehumanitária', 
    'Crisehumanitária', 
    'CriseHumanitária', 
    'CRISEHUMANITÁRIA', 
    'crisevenezuelana', 
    'Crisevenezuelana', 
    'CriseVenezuelana', 
    'CRISEVENEZUELANA', 
    'Discriminação', 
    'discriminação', 
    'DISCRIMINAÇÃO', 
    'estereótipo', 
    'Estereótipo', 
    'ESTEREÓTIPO', 
    'fronteira', 
    'Fronteira', 
    'FRONTEIRA', 
    'migrantes', 
    'Migrantes', 
    'MIGRANTES', 
    'preconceito', 
    'Preconceito', 
    'PRECONCEITO', 
    'refugiados', 
    'Refugiados', 
    'REFUGIADOS', 
    'roraizuela', 
    'Roraizuela', 
    'RORAIZUELA', 
    'venebrasil', 
    'Venebrasil', 
    'VeneBrasil', 
    'VENEBRASIL', 
    'venezuelanosnobrasil', 
    'Venezuelanosnobrasil', 
    'VenezuelanosNoBrasil', 
    'VENEZUELANOSNOBRASIL', 
    'venezuraima', 
    'Venezuraima', 
    'VENEZURAIMA', 
    'violência', 
    'Violência', 
    'VIOLÊNCIA', 
    'xenofobia', 
    'Xenofobia', 
    'XENOFOBIA'
]

expressions = [
    'ameaça', 
    'aporofobia', 
    'carga', 
    'conflito', 
    'crise', 
    'delinquência', 
    'desconfiança', 
    'desemprego', 
    'desigualdade', 
    'desordem', 
    'direitos', 
    'discriminação', 
    'estigma', 
    'estrangeiro', 
    'exclusão', 
    'fronteira',  
    'hostilidade', 
    'humanitário', 
    'identidade', 
    'inferior', 
    'intolerância', 
    'invasão', 
    'marginal', 
    'perigo', 
    'preconceito', 
    'problema', 
    'racismo', 
    'refugiada', 
    'refugiado', 
    'refugiados', 
    'rejeição', 
    'roraizuela', 
    'venebrasil', 
    'venezuela', 
    'venezuelana', 
    'venezuelanas', 
    'venezuelano', 
    'venezuelanos', 
    'venezuraima', 
    'violência', 
    'xenofobia'
]

# Create a filtered DataFrame
filtered_tweets_spark_df = tweets_spark_df.filter(
    array_contains('entities.hashtags.text', hashtags[0]) |\
    array_contains('entities.hashtags.text', hashtags[1]) |\
    array_contains('entities.hashtags.text', hashtags[2]) |\
    array_contains('entities.hashtags.text', hashtags[3]) |\
    array_contains('entities.hashtags.text', hashtags[4]) |\
    array_contains('entities.hashtags.text', hashtags[5]) |\
    array_contains('entities.hashtags.text', hashtags[6]) |\
    array_contains('entities.hashtags.text', hashtags[7]) |\
    array_contains('entities.hashtags.text', hashtags[8]) |\
    array_contains('entities.hashtags.text', hashtags[9]) |\
    array_contains('entities.hashtags.text', hashtags[10]) |\
    array_contains('entities.hashtags.text', hashtags[11]) |\
    array_contains('entities.hashtags.text', hashtags[12]) |\
    array_contains('entities.hashtags.text', hashtags[13]) |\
    array_contains('entities.hashtags.text', hashtags[14]) |\
    array_contains('entities.hashtags.text', hashtags[15]) |\
    array_contains('entities.hashtags.text', hashtags[16]) |\
    array_contains('entities.hashtags.text', hashtags[17]) |\
    array_contains('entities.hashtags.text', hashtags[18]) |\
    array_contains('entities.hashtags.text', hashtags[19]) |\
    array_contains('entities.hashtags.text', hashtags[20]) |\
    array_contains('entities.hashtags.text', hashtags[21]) |\
    array_contains('entities.hashtags.text', hashtags[22]) |\
    array_contains('entities.hashtags.text', hashtags[23]) |\
    array_contains('entities.hashtags.text', hashtags[24]) |\
    array_contains('entities.hashtags.text', hashtags[25]) |\
    array_contains('entities.hashtags.text', hashtags[26]) |\
    array_contains('entities.hashtags.text', hashtags[27]) |\
    array_contains('entities.hashtags.text', hashtags[28]) |\
    array_contains('entities.hashtags.text', hashtags[29]) |\
    array_contains('entities.hashtags.text', hashtags[30]) |\
    array_contains('entities.hashtags.text', hashtags[31]) |\
    array_contains('entities.hashtags.text', hashtags[32]) |\
    array_contains('entities.hashtags.text', hashtags[33]) |\
    array_contains('entities.hashtags.text', hashtags[34]) |\
    array_contains('entities.hashtags.text', hashtags[35]) |\
    array_contains('entities.hashtags.text', hashtags[36]) |\
    array_contains('entities.hashtags.text', hashtags[37]) |\
    array_contains('entities.hashtags.text', hashtags[38]) |\
    array_contains('entities.hashtags.text', hashtags[39]) |\
    array_contains('entities.hashtags.text', hashtags[40]) |\
    array_contains('entities.hashtags.text', hashtags[41]) |\
    array_contains('entities.hashtags.text', hashtags[42]) |\
    array_contains('entities.hashtags.text', hashtags[43]) |\
    array_contains('entities.hashtags.text', hashtags[44]) |\
    array_contains('entities.hashtags.text', hashtags[45]) |\
    array_contains('entities.hashtags.text', hashtags[46]) |\
    array_contains('entities.hashtags.text', hashtags[47]) |\
    array_contains('entities.hashtags.text', hashtags[48]) |\
    array_contains('entities.hashtags.text', hashtags[49]) |\
    array_contains('entities.hashtags.text', hashtags[50]) |\
    array_contains('entities.hashtags.text', hashtags[51]) |\
    array_contains('entities.hashtags.text', hashtags[52]) |\
    array_contains('entities.hashtags.text', hashtags[53]) |\
    array_contains('entities.hashtags.text', hashtags[54]) |\
    array_contains('entities.hashtags.text', hashtags[55]) |\
    lower(col('text')).contains(expressions[0]) |\
    lower(col('text')).contains(expressions[1]) |\
    lower(col('text')).contains(expressions[2]) |\
    lower(col('text')).contains(expressions[3]) |\
    lower(col('text')).contains(expressions[4]) |\
    lower(col('text')).contains(expressions[5]) |\
    lower(col('text')).contains(expressions[6]) |\
    lower(col('text')).contains(expressions[7]) |\
    lower(col('text')).contains(expressions[8]) |\
    lower(col('text')).contains(expressions[9]) |\
    lower(col('text')).contains(expressions[10]) |\
    lower(col('text')).contains(expressions[11]) |\
    lower(col('text')).contains(expressions[12]) |\
    lower(col('text')).contains(expressions[13]) |\
    lower(col('text')).contains(expressions[14]) |\
    lower(col('text')).contains(expressions[15]) |\
    lower(col('text')).contains(expressions[16]) |\
    lower(col('text')).contains(expressions[17]) |\
    lower(col('text')).contains(expressions[18]) |\
    lower(col('text')).contains(expressions[19]) |\
    lower(col('text')).contains(expressions[20]) |\
    lower(col('text')).contains(expressions[21]) |\
    lower(col('text')).contains(expressions[22]) |\
    lower(col('text')).contains(expressions[23]) |\
    lower(col('text')).contains(expressions[24]) |\
    lower(col('text')).contains(expressions[25]) |\
    lower(col('text')).contains(expressions[26]) |\
    lower(col('text')).contains(expressions[27]) |\
    lower(col('text')).contains(expressions[28]) |\
    lower(col('text')).contains(expressions[29]) |\
    lower(col('text')).contains(expressions[30]) |\
    lower(col('text')).contains(expressions[31]) |\
    lower(col('text')).contains(expressions[32]) |\
    lower(col('text')).contains(expressions[33]) |\
    lower(col('text')).contains(expressions[34]) |\
    lower(col('text')).contains(expressions[35]) |\
    lower(col('text')).contains(expressions[36]) |\
    lower(col('text')).contains(expressions[37]) |\
    lower(col('text')).contains(expressions[38]) |\
    lower(col('text')).contains(expressions[39]) |\
    lower(col('text')).contains(expressions[40])
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `entities`.`hashtags`.`text` cannot be resolved. Did you mean one of the following? [`data`, `errors`, `includes`].;
'Filter ((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((((array_contains('entities.hashtags.text, acolhimento) OR array_contains('entities.hashtags.text, Acolhimento)) OR array_contains('entities.hashtags.text, ACOLHIMENTO)) OR array_contains('entities.hashtags.text, aporofobia)) OR array_contains('entities.hashtags.text, Aporofobia)) OR array_contains('entities.hashtags.text, APOROFOBIA)) OR array_contains('entities.hashtags.text, brasilvaivirarvenezuela)) OR array_contains('entities.hashtags.text, Brasilvaivirarvenezuela)) OR array_contains('entities.hashtags.text, BrasilVaiVirarVenezuela)) OR array_contains('entities.hashtags.text, BRASILVAIVIRARVENEZUELA)) OR array_contains('entities.hashtags.text, crisehumanit?ria)) OR 